# SET UP

## CREAR EL ENTORNO PARA EL PROYECTO

Crear un nuevo entorno e instalar los paquetes básicos:

1. Ir al prompt o terminal
2. Copiar y poner la línea inferior:

conda create --name retail python numpy pandas matplotlib seaborn scikit-learn scipy sqlalchemy xgboost jupyter

3. Activar el nuevo entorno:

conda activate retail

4. Instalar los paquetes que están en otros canales:

conda install -c conda-forge pyjanitor scikit-plot yellowbrick imbalanced-learn jupyter_contrib_nbextensions cloudpickle

conda install -c districtdatalabs yellowbrick

pip install category_encoders

5. Crear el environment.yml

conda env export > retail.yml


## IMPORTAR PAQUETES

In [ ]:
import os
import numpy as np
import pandas as pd

#Automcompletar rápido
%config IPCompleter.greedy=True

## CREAR EL DIRECTORIO DEL PROYECTO

### Definir el directorio raiz donde vamos a crear el proyecto

In [ ]:
raiz = r'C:\Users\mcent\OneDrive\Escritorio\MARTIN\MASTERS\DS4B\DATA SCIENCE\03_MACHINE_LEARNING\07_CASOS'

### Definir el nombre del proyecto

In [ ]:
nombre_dir = '\FORECASTING RETAIL'

### Crear el directorio y la estructura del proyecto

In [ ]:
path = raiz + nombre_dir

In [ ]:
# try:
#     os.mkdir(path)
#     os.mkdir(path + '/01_Documentos')
#     os.mkdir(path + '/02_Datos')
#     os.mkdir(path + '/02_Datos/01_Originales')
#     os.mkdir(path + '/02_Datos/02_Validacion')
#     os.mkdir(path + '/02_Datos/03_Trabajo')
#     os.mkdir(path + '/02_Datos/04_Caches')
#     os.mkdir(path + '/03_Notebooks')
#     os.mkdir(path + '/03_Notebooks/01_Funciones')
#     os.mkdir(path + '/03_Notebooks/02_Desarrollo')
#     os.mkdir(path + '/03_Notebooks/03_Sistema')
#     os.mkdir(path + '/04_Modelos')
#     os.mkdir(path + '/05_Resultados')
#     os.mkdir(path + '/09_Otros')
    
# except OSError:
#     print ("La creación del directorio %s ha fallado" % path)
# else:
#     print ("Se ha creado satisfactoriamente el directorio %s " % path)

### Situar el directorio de trabajo en el proyecto

In [ ]:
os.chdir(path)

### Mover el environment.yml

Buscar manualmente el archivo retail.yml, generalmente ubicado en C:\Users\tu_usuario y moverlo a la carpeta /01_Documentos'

## CREAR LOS DATASETS INICIALES

Poner los datos originales en la carpeta '/02_Datos/01_Originales'

### Importar los datos

Sustituir el nombre_fichero_datos.

In [ ]:
nombre_fichero_datos = 'hipermercado.db'
ruta_completa = path + '/02_Datos/01_Originales/' + nombre_fichero_datos

Cargar los datos.

In [ ]:
import sqlalchemy as sa

In [ ]:
con = sa.create_engine('sqlite:///' + ruta_completa)

In [15]:
from sqlalchemy import inspect
insp = inspect(con)
tablas = insp.get_table_names()
tablas

['calendar', 'sales', 'sell_prices']

In [12]:
cal = pd.read_sql('calendar', con)
cal.drop(columns='index',inplace=True)
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2
0,2013-01-01,11249,Tuesday,4,1,2013,d_704,NewYear,National,None,None
1,2013-01-02,11249,Wednesday,5,1,2013,d_705,None,None,None,None
2,2013-01-03,11249,Thursday,6,1,2013,d_706,None,None,None,None
3,2013-01-04,11249,Friday,7,1,2013,d_707,None,None,None,None
4,2013-01-05,11250,Saturday,1,1,2013,d_708,None,None,None,None


In [31]:
sales = pd.read_sql('sales', con)
sales.drop(columns='index',inplace=True)
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_704,d_705,d_706,d_707,...,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,0,224,241,232,...,5,2,0,0,6,0,6,0,0,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,54,63,44,0,65,90,104,73,76,97
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,20,23,23,25,...,43,40,39,0,29,33,27,13,26,47
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,34,27,40,49,...,31,43,32,0,52,37,32,29,34,27
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,29,45,28,0,46,36,40,31,46,36


In [14]:
prices = pd.read_sql('sell_prices', con)
prices.drop(columns='index',inplace=True)
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,FOODS_3_090,11249,1.25
1,CA_3,FOODS_3_090,11250,1.25
2,CA_3,FOODS_3_090,11251,1.25
3,CA_3,FOODS_3_090,11252,1.25
4,CA_3,FOODS_3_090,11301,1.38


## CREAR EL TABLON ANALITICO

### Entender la relación entre las tablas

NOTAS:

* Primero unir cal con sales, ya que para unir prices necesitamos campos de ambas
* Para unir cal con sales primero hay que modificar sales, para pasar la variable d a filas

### Unión de cal con sales

#### Transformar la estructura de sales

In [33]:
sales = sales.melt(id_vars =sales.columns[0:6], 
                   var_name ='d', 
                   value_name='ventas')
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d,ventas
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0
...,...,...,...,...,...,...,...,...
21895,FOODS_3_329_CA_4_validation,FOODS_3_329,FOODS_3,FOODS,CA_4,CA,d_1798,6
21896,FOODS_3_555_CA_4_validation,FOODS_3_555,FOODS_3,FOODS,CA_4,CA,d_1798,5
21897,FOODS_3_586_CA_4_validation,FOODS_3_586,FOODS_3,FOODS,CA_4,CA,d_1798,11
21898,FOODS_3_587_CA_4_validation,FOODS_3_587,FOODS_3,FOODS,CA_4,CA,d_1798,16


In [35]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,ventas
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0


In [ ]:
sales = sales.melt(id_vars = sales.columns[0:6],
                   var_name = 'd',
                   value_name = 'ventas')
sales

Eliminamos la variable id que ya no aporta nada.

In [36]:
sales.drop(columns='id',inplace=True)

#### Cruzar con calendar para incorporar su información

In [44]:
df = pd.merge(left = sales, right=cal, on ='d',how = 'left')
df

,item_id,dept_id,cat_id,store_id,state_id,d,ventas,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21895,FOODS_3_329,FOODS_3,FOODS,CA_4,CA,d_1798,6,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None
21896,FOODS_3_555,FOODS_3,FOODS,CA_4,CA,d_1798,5,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None
21897,FOODS_3_586,FOODS_3,FOODS,CA_4,CA,d_1798,11,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None
21898,FOODS_3_587,FOODS_3,FOODS,CA_4,CA,d_1798,16,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None


In [45]:
pd.crosstab(df.event_name_1,df.event_name_2)

event_name_2,Cinco De Mayo,Father's day,OrthodoxEaster
event_name_1,,,
Easter,0,0,20
NBAFinalsEnd,0,20,0
OrthodoxEaster,20,0,0


In [55]:
df.loc[df.event_name_1 == 'OrthodoxEaster'].date.unique()

array(['2013-05-05', '2015-04-12'], dtype=object)

In [ ]:
df = sales.merge(right = cal, how = 'left', on = 'd')
df.head()

### Incorporacion de prices

In [56]:
df.head(3)

,item_id,dept_id,cat_id,store_id,state_id,d,ventas,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None


In [58]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,FOODS_3_090,11249,1.25
1,CA_3,FOODS_3_090,11250,1.25
2,CA_3,FOODS_3_090,11251,1.25
3,CA_3,FOODS_3_090,11252,1.25
4,CA_3,FOODS_3_090,11301,1.38


In [60]:
df = pd.merge(left = df, right = prices, on = ['item_id','store_id','wm_yr_wk'], how = 'left')
df

,item_id,dept_id,cat_id,store_id,state_id,d,ventas,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.25
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.98
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.28
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.48
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21895,FOODS_3_329,FOODS_3,FOODS,CA_4,CA,d_1798,6,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,2.98
21896,FOODS_3_555,FOODS_3,FOODS,CA_4,CA,d_1798,5,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,1.68
21897,FOODS_3_586,FOODS_3,FOODS,CA_4,CA,d_1798,11,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,1.68
21898,FOODS_3_587,FOODS_3,FOODS,CA_4,CA,d_1798,16,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,2.48


Comprobamos.

In [61]:
df[['store_id','item_id','wm_yr_wk','d','sell_price']]\
    .sort_values(by = ['store_id','item_id','wm_yr_wk','d'])\
    .head(10)

,store_id,item_id,wm_yr_wk,d,sell_price
0,CA_3,FOODS_3_090,11249,d_704,1.25
20,CA_3,FOODS_3_090,11249,d_705,1.25
40,CA_3,FOODS_3_090,11249,d_706,1.25
60,CA_3,FOODS_3_090,11249,d_707,1.25
80,CA_3,FOODS_3_090,11250,d_708,1.25
100,CA_3,FOODS_3_090,11250,d_709,1.25
120,CA_3,FOODS_3_090,11250,d_710,1.25
140,CA_3,FOODS_3_090,11250,d_711,1.25
160,CA_3,FOODS_3_090,11250,d_712,1.25
180,CA_3,FOODS_3_090,11250,d_713,1.25


### Reordenar y poner date como index

In [62]:
df.head(1)

,item_id,dept_id,cat_id,store_id,state_id,d,ventas,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.25


In [63]:
orden = ['date',
         'state_id',
         'store_id',
         'dept_id',
         'cat_id',
         'item_id',
         'wm_yr_wk',
         'd',
         'ventas',
         'sell_price',
         'year',
         'month',
         'wday',
         'weekday',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2']

In [64]:
df = df[orden].set_index('date')
df

,state_id,store_id,dept_id,cat_id,item_id,wm_yr_wk,d,ventas,sell_price,year,month,wday,weekday,event_name_1,event_type_1,event_name_2,event_type_2
date,,,,,,,,,,,,,,,,,
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_090,11249,d_704,0,1.25,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_120,11249,d_704,0,4.98,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_202,11249,d_704,20,4.28,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_252,11249,d_704,34,1.48,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_288,11249,d_704,0,NaN,2013,1,4,Tuesday,NewYear,National,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_329,11548,d_1798,6,2.98,2015,12,6,Thursday,None,None,None,None
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_555,11548,d_1798,5,1.68,2015,12,6,Thursday,None,None,None,None
2015-12-31,CA,CA_4,FOODS_3,FOODS,FOODS_3_586,11548,d_1798,11,1.68,2015,12,6,Thursday,None,None,None,None


### Extraer y reservar el dataset de validación

Dado que es un problema de forecasting no podemos extraer el dataset de validación de forma aleatoria.

Si no que éste estará compuesto por los últimos datos disponibles.

In [76]:
df.index.max()

'2015-12-31'

In [77]:
nombre_fichero_validacion = 'validacion.csv'

ruta_completa = path + '/02_Datos/02_Validacion/' + nombre_fichero_validacion

val.to_csv(ruta_completa)

In [78]:
val = df.loc['2015-12-01':'2015-12-31']

### Extraer y guardar el dataset de trabajo

In [79]:
trabajo = df.loc[~df.index.isin(val.index)]

In [80]:
nombre_fichero_trabajo = 'trabajo.csv'

ruta_completa = path + '/02_Datos/03_Trabajo/' + nombre_fichero_trabajo

trabajo.to_csv(ruta_completa)

### Extraer y guardar una muestra (opcional)

En este proyecto no haremos muestra.